In [1]:
# !mamba install -c conda-forge geopandas
# !mamba install -c conda-forge geopandas fiona gdal pyogrio

In [2]:
from helpers.locations import *
from helpers.drugs import *
from helpers.dates import *
from helpers.population import CountyNormalizer
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from PIL import Image
import glob

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import numpy as np

In [3]:
state = "Colorado"
legalisation_date = "2014-01-01"

county_normalizer = CountyNormalizer()

# df_drug = pd.read_csv("data/drugs/US_Oxycodone.csv")
DRUG_NAMES = ["US_Amphetamine", "US_Codeine", "US_Hydrocodone", "US_Morphine", "US_Oxycodone"]
df_drug = get_drugs_dfs(DRUG_NAMES)
df_drug = df_drug.generate_state_column("placeDcid")
df_drug = df_drug.generate_year()
df_drug['FIPS'] = df_drug['placeDcid'].str.split('/').str[-1]
df_drug["population"] = df_drug.apply(county_normalizer.get_population, axis=1)
df_drug.dropna(inplace= True)
df_drug

,placeDcid,placeName,Date,US_Amphetamine,US_Codeine,US_Hydrocodone,US_Morphine,US_Oxycodone,state,Year,FIPS,population
0,geoId/01001,Autauga County,2006-01,301.62,456.09,1484.79,472.42,720.13,Alabama,2006,01001,49105.0
1,geoId/01001,Autauga County,2006-04,277.00,500.07,1641.61,535.16,776.14,Alabama,2006,01001,49105.0
2,geoId/01001,Autauga County,2006-07,280.13,442.30,1660.31,459.76,707.36,Alabama,2006,01001,49105.0
3,geoId/01001,Autauga County,2006-10,323.46,425.44,1977.05,487.07,769.49,Alabama,2006,01001,49105.0
4,geoId/01001,Autauga County,2007-01,341.98,447.23,1989.93,528.35,763.86,Alabama,2007,01001,49834.0
...,...,...,...,...,...,...,...,...,...,...,...,...
150763,geoId/56045,Weston County,2016-10,59.48,45.19,103.31,42.03,159.98,Wyoming,2016,56045,7228.0
150764,geoId/56045,Weston County,2017-01,58.34,46.35,100.78,43.06,153.44,Wyoming,2017,56045,6962.0
150765,geoId/56045,Weston County,2017-04,54.30,32.44,95.51,52.34,154.04,Wyoming,2017,56045,6962.0
150766,geoId/56045,Weston County,2017-07,62.98,36.01,91.86,37.70,142.69,Wyoming,2017,56045,6962.0


In [4]:
# Loop through each drug column and divide by 'population' column
for drug in DRUG_NAMES:
    df_drug[drug] = df_drug[drug] / df_drug['population']
df_drug

,placeDcid,placeName,Date,US_Amphetamine,US_Codeine,US_Hydrocodone,US_Morphine,US_Oxycodone,state,Year,FIPS,population
0,geoId/01001,Autauga County,2006-01,0.006142,0.009288,0.030237,0.009621,0.014665,Alabama,2006,01001,49105.0
1,geoId/01001,Autauga County,2006-04,0.005641,0.010184,0.033431,0.010898,0.015806,Alabama,2006,01001,49105.0
2,geoId/01001,Autauga County,2006-07,0.005705,0.009007,0.033811,0.009363,0.014405,Alabama,2006,01001,49105.0
3,geoId/01001,Autauga County,2006-10,0.006587,0.008664,0.040262,0.009919,0.015670,Alabama,2006,01001,49105.0
4,geoId/01001,Autauga County,2007-01,0.006862,0.008974,0.039931,0.010602,0.015328,Alabama,2007,01001,49834.0
...,...,...,...,...,...,...,...,...,...,...,...,...
150763,geoId/56045,Weston County,2016-10,0.008229,0.006252,0.014293,0.005815,0.022133,Wyoming,2016,56045,7228.0
150764,geoId/56045,Weston County,2017-01,0.008380,0.006658,0.014476,0.006185,0.022040,Wyoming,2017,56045,6962.0
150765,geoId/56045,Weston County,2017-04,0.007799,0.004660,0.013719,0.007518,0.022126,Wyoming,2017,56045,6962.0
150766,geoId/56045,Weston County,2017-07,0.009046,0.005172,0.013194,0.005415,0.020496,Wyoming,2017,56045,6962.0


In [5]:
# df_drug_state = df_drug[df_drug["state"] == state]
# df_drug_state

In [6]:
# # Example legalisation_date
# # legalisation_date_ts = pd.Timestamp("2014-01-01")  # Replace with actual date

# DATE_COLUMN = "Date"

# # Split into two data frames
# df_before_legalisation = df_drug_state[df_drug_state[DATE_COLUMN] < legalisation_date]
# df_after_legalisation = df_drug_state[df_drug_state[DATE_COLUMN] >= legalisation_date]
# df_before_legalisation

# Visualisations

In [7]:
# ORIGINAL_COL = "US_Codeine"
# GRAPH_COL = ORIGINAL_COL+"_Log"
# df_drug[GRAPH_COL] = np.log(df_drug[ORIGINAL_COL])

# # Create dataframe and process FIPS codes
# cropped_df = df_drug[df_drug["Date"] == "2017-10"]

# # Load US counties shapefile
# counties = gpd.read_file('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json')
# merged = counties.merge(cropped_df, how='left', left_on='id', right_on='FIPS')

# # Create the plot
# fig, ax = plt.subplots(1, figsize=(15, 10))

# # Create choropleth map
# merged.plot(
#     column=GRAPH_COL,
#     ax=ax,
#     legend=True,
#     legend_kwds={'label': f'{GRAPH_COL} Usage'},
#     missing_kwds={'color': 'lightgrey'},
#     cmap='YlOrRd'
# )

# # Set the map boundaries to focus on continental US
# ax.set_xlim([-125, -66.5])  # Longitude limits for continental US
# ax.set_ylim([24, 49])      # Latitude limits for continental US

# # Customize the map
# plt.title('Codeine Usage by County (2014)', pad=20)
# plt.axis('off')

# # Show the plot
# plt.show()

In [14]:
import os
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

SOURCE_NAME = "US_Oxycodone"

def generate_images(name):
    ORIGINAL_COL = name  # e.g., "US_Codeine"
    GRAPH_COL = ORIGINAL_COL + "_Log"
    
    # Apply log transformation
    df_drug[GRAPH_COL] = np.log(df_drug[ORIGINAL_COL] + 1e-10)  # Adding small constant to avoid log(0)

    # Create base output directory for maps
    base_output_dir = Path('drug_maps')
    base_output_dir.mkdir(exist_ok=True)
    
    # Create drug-specific directory
    drug_output_dir = base_output_dir / ORIGINAL_COL.lower()
    drug_output_dir.mkdir(exist_ok=True)

    # Get sorted list of unique dates
    dates = sorted(df_drug['Date'].unique())

    # Find the global min and max for consistent color scale
    global_min = df_drug[GRAPH_COL].min()
    global_max = df_drug[GRAPH_COL].max()

    # Load US counties shapefile (load once outside the loop for efficiency)
    counties = gpd.read_file('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json')

    # Create a map for each date
    for date in dates:
        # Filter dataframe for this date
        cropped_df = df_drug[df_drug["Date"] == date]
        
        # Merge with geographic data
        merged = counties.merge(cropped_df, how='left', left_on='id', right_on='FIPS')
        
        # Create the plot
        fig, ax = plt.subplots(1, figsize=(15, 10))
        
        # Create choropleth map
        merged.plot(
            column=GRAPH_COL,
            ax=ax,
            legend=True,
            legend_kwds={'label': f'{GRAPH_COL} Usage'},
            missing_kwds={'color': 'lightgrey'},
            cmap='YlOrRd',
            vmin=global_min,  # Set consistent color scale
            vmax=global_max
        )
        
        # Set the map boundaries to focus on continental US
        ax.set_xlim([-125, -66.5])
        ax.set_ylim([24, 49])
        
        # Customize the map
        plt.title(f'{GRAPH_COL} Usage by County ({date})', pad=20)
        plt.axis('off')
        
        # Save the plot
        filename = drug_output_dir / f'{ORIGINAL_COL.lower()}_map_{date.replace("-", "_")}.png'
        plt.savefig(filename, bbox_inches='tight', dpi=300)
        plt.close()  # Close the figure to free memory

    print(f"Created maps for {len(dates)} dates in the '{drug_output_dir}' directory")

# Example usage
generate_images(SOURCE_NAME)

Created maps for 48 dates in the 'drug_maps/us_oxycodone' directory


In [15]:
def generate_gif(name: str):
    lower_name = name.lower()
    path = f"drug_maps/{lower_name}/{lower_name}_map_*.png"

    try:
        # Get list of saved images
        images = []
        files = sorted(glob.glob(path))
        
        if not files:
            print(f"No images found for {name}. Check the path: {path}")
            return

        # Load all images
        for filename in files:
            images.append(Image.open(filename))
        
        # Create output directory if it doesn't exist
        output_dir = Path(f"drug_maps/{lower_name}")
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Save as GIF
        output_path = output_dir / f"{lower_name}_usage_animation.gif"
        images[0].save(
            output_path,
            save_all=True,
            append_images=images[1:],
            duration=500,  # Duration for each frame in milliseconds
            loop=0
        )
        print(f"Created animation: {output_path}")
    except ImportError:
        print("PIL (Pillow) library not installed. Skipping GIF creation.")
    except FileNotFoundError as e:
        print(f"File not found: {e}")

# Example usage
generate_gif(SOURCE_NAME)

Created animation: drug_maps/us_oxycodone/us_oxycodone_usage_animation.gif
